<a href="https://colab.research.google.com/github/akshatshah91/Game-AI/blob/master/Monte_Carlo_Tree_Search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import gym
import random
import math
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torch import randint
import torch.nn.functional as F
import matplotlib.pyplot as plt
from IPython.display import clear_output
from collections import deque
import sys
import numpy as np
import copy